# Глава 21 Сохранение и загрузка натренированных моделей(МО Крис Элбон)

Нам нужно интегрировать нашу модель с существующим программным приложением.

Мы должны уметь сохранять наши модели после тренировки и загружать их, когда они необходимы в приложении.

# Сохранение и загрузка модели scikit-learn

Дана натренированная модель scikit-learn, и требуется ее сохранить  и загрузить в другом месте.

Сохранить модель в качестве файла консервации:

In [1]:
#загрузить библиотеки
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.externals import joblib

C:\Users\user\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
#загрузить данные
iris = datasets.load_iris()
features = iris.data
target = iris.target

In [3]:
#создать объект классификатор случайных лесов
classifer = RandomForestClassifier()

In [4]:
#натренировать модель
model = classifer.fit(features, target)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [5]:
#сохранить модель с помощью файла консервации
joblib.dump(model, "model.pkl")

['model.pkl']

После того, как модель будет сохранена, мы можем использовать библиотеку scikit-learn в нашем конечном приложении (например, в веб-приложении), чтобы загрузить модель:

In [6]:
#загрузить модель из файла
classifer = joblib.load("model.pkl")

И использовать ее для выполнения предсказаний:

In [7]:
#создать новое наблюдение
new_observation = [[5.2, 3.2, 1.1, 0.1]]

In [8]:
#предсказать класс наблюдения
classifer.predict(new_observation)

array([0])

Первым шагом при использовании модели в производственной среде является сохранение этой модели  в виде файла, который может быть загружен другим приложением или рабочим процессом.

Мы можем сделать это, сохранив модель в виде файла pickle, специфического для Python формата встроенной одноименной библиотеки для консервации данных.

В частности, для сохранения модели мы используем библиотеку joblib, которая расширяет библиотеку pickle для случаев, когда имеются большие массивы NumPy  - распространенное явление для моделей, натренированных в библиотеке scikit-learn.

При сохранении моделей scikit-learn имейте ввиду, что сохраненные модели могут быть несовместимы между версиями scikit-learn, поэтому может быть полезно включить используемую в модели версию библиотеки scikit-learn в имя файла:

In [9]:
#импортировать библиотеку
import sklearn

In [13]:
#получить версию библиотеки skikit-learn
scikit_version = sklearn.__version__

In [14]:
#сохранить модель в качестве файла консервации
joblib.dump(model, "model_{version}.pkl".format(version=scikit_version))

['model_0.21.3.pkl']

# Сохранение и загрузка модели Keras

Дана натренированная модель Keras, и требуется ее сохранить и загрузить в другом месте.

Сохранить модель в файле формата HDF5:

In [15]:
#загрузить библиотеки
import numpy as np
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import  layers
from keras.models import load_model

In [16]:
#задать начальное значение для генератора псевдослучайных чисел
np.random.seed(0)

In [17]:
#задать желаемое количество признаков
number_of_features = 1000

In [18]:
#загрузить данные и вектор из набора данных IMDB о кинофильмах
(data_train, target_train), (data_test, target_test) = imdb.load_data(num_words=number_of_features)

In [19]:
#конвертировать данные IMBD о кинофильмах 
#в матрицу признаков в кодировке с одним активным состоянием
tokenizer = Tokenizer(num_words=number_of_features)
features_train = tokenizer.sequences_to_matrix(data_train, mode="binary")
features_test = tokenizer.sequences_to_matrix(data_test, mode="binary")

In [20]:
#инициализировать нейронную сеть
network = models.Sequential()

In [21]:
#добавить полносвязный слой с активационной функцией ReLU
network.add(layers.Dense(
    units=16,
    activation="relu",
    input_shape=(number_of_features,)
)) 

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
#добавить полносвязный слой с сигмоидальной активационной функцией
network.add(layers.Dense(
    units=1,
    activation="sigmoid"
)) 

In [23]:
#скомпилировать нейронную сеть
network.compile(
    loss="binary_crossentropy", #перекрестная энтропия
    optimizer="rmsprop", #распространение среднеквадратичной ошибки
    metrics=["accuracy"] #точностный показатель результтивности
)

In [24]:
#натренировать нейронную сеть
history = network.fit(
    features_train, #признаки
    target_train,  #вектор целей
    epochs=3,  #количество эпох
    verbose=0, #печатать описание после каждой эпохи
    batch_size=100,  #количество наблюдений на пакет
    validation_data=(features_test, target_test) #тестовые данные
)

Instructions for updating:
Use tf.cast instead.


In [25]:
#сохранить нейронную сеть
network.save("model.h5")

Затем мы можем загрузить модель в другом приложении либо для дополнительной тренировки:

In [26]:
#загрузить нейронную сеть
network = load_model("model.h5")

В отличие от библиотеки scikit-learn, в библиотеке Keras не рекомендуется сохранять модели с помощью встроенной библиотеке pickle. Вместо этого модели сохраняются как файл HDF5. 

Файл HDF5 содержит все необходимое  для того, чтобы не только загрузить модель для выполнения предсказаний (те архитектуру и натренированные параметры), но и перезапустить процесс тренировки (те настройки потери и оптимизатора и текущее состояние).